### 基礎的な操作

ssh で 192.168.99.199 で接続してください(openstack/openstack)

root ユーザのホームディレクトリには OpenStack を管理者ユーザーで使うための設定ファイルが保存されています。

openstack ユーザのホームディレクトリには OpenStack を一般ユーザで使うための設定ファイルが保存されています。

#### 認証情報の読み込み（openstackユーザで実施しています）
openstackユーザのホームディレクトリの openrc には演習用環境を利用するためのOpenStack認証情報が記載されています。各種コマンドで操作を行う前に、このファイルを読み込んで、環境変数へ認証情報を設定しておきます。

In [3]:
cat ~/openrc

export OS_AUTH_URL=http://192.168.99.199:5000/v2.0
export OS_REGION_NAME=RegionOne
export OS_TENANT_NAME=project01
export OS_USERNAME=openstack
export OS_PASSWORD=openstack



In [4]:
source ~/openrc

In [5]:
env | grep OS_

OS_REGION_NAME=RegionOne
OS_PASSWORD=openstack
OS_AUTH_URL=http://192.168.99.199:5000/v2.0
OS_USERNAME=openstack
OS_TENANT_NAME=project01


#### 初期状態の確認（一般ユーザーの初期状態）
初期状態では、管理者が設定する public ネットワーク、Cirrosイメージ作成されています。
みなさんがユーザとしてOpenStackを使う場合は、この状態からのスタートになります。

VMはありません

In [6]:
nova list

+----+------+--------+------------+-------------+----------+
| ID | Name | Status | Task State | Power State | Networks |
+----+------+--------+------------+-------------+----------+
+----+------+--------+------------+-------------+----------+


管理者が作成した public ネットワークのみが存在しています。

In [7]:
neutron net-list

+----------------------------------+--------+----------------------------------+
| id                               | name   | subnets                          |
+----------------------------------+--------+----------------------------------+
| 17e4bba8-2a15-46fb-              | public | a271b0a1-c912-492c-858e-         |
| ab54-fb463aee503c                |        | d5bd41888246                     |
+----------------------------------+--------+----------------------------------+


今回の演習用に作成したキーペアが作成されています。

In [8]:
nova keypair-list

+--------+------+-------------------------------------------------+
| Name   | Type | Fingerprint                                     |
+--------+------+-------------------------------------------------+
| my-key | ssh  | 43:0f:f5:b3:3e:ab:b7:1b:6a:a9:fc:65:4a:e6:42:89 |
+--------+------+-------------------------------------------------+


テスト用のOSイメージが登録されています。

In [9]:
openstack image list

+--------------------------------------+--------------+--------+
| ID                                   | Name         | Status |
+--------------------------------------+--------------+--------+
| 66722486-ac79-4cd2-9a7d-c2479a9f6f92 | cirros-0.3.5 | active |
+--------------------------------------+--------------+--------+


フレーバー（AWSのインスタンスタイプ）が予めいくつか定義されています。これは管理者が設定します。

In [10]:
nova flavor-list

+----+-----------+-----------+------+-----------+------+-------+-------------+-----------+
| ID | Name      | Memory_MB | Disk | Ephemeral | Swap | VCPUs | RXTX_Factor | Is_Public |
+----+-----------+-----------+------+-----------+------+-------+-------------+-----------+
| 1  | m1.tiny   | 512       | 1    | 0         |      | 1     | 1.0         | True      |
| 2  | m1.small  | 2048      | 20   | 0         |      | 1     | 1.0         | True      |
| 3  | m1.medium | 4096      | 40   | 0         |      | 2     | 1.0         | True      |
| 4  | m1.large  | 8192      | 80   | 0         |      | 4     | 1.0         | True      |
| 5  | m1.xlarge | 16384     | 160  | 0         |      | 8     | 1.0         | True      |
+----+-----------+-----------+------+-----------+------+-------+-------------+-----------+


管理的なファイアーウォール機能が予め２つ定義されています。default は初期状態で存在し、すべての通信（→VM）を遮断します。ここでは open-all という全ての通信を許可するルールを演習用に作成しています。

In [11]:
neutron security-group-list

+---------------------------------+----------+---------------------------------+
| id                              | name     | security_group_rules            |
+---------------------------------+----------+---------------------------------+
| 43a9ef6f-2cf2-484f-a37a-        | default  | egress, IPv4                    |
| d45a290ac5db                    |          | egress, IPv6                    |
|                                 |          | ingress, IPv4, remote_group_id: |
|                                 |          | 43a9ef6f-2cf2-484f-a37a-        |
|                                 |          | d45a290ac5db                    |
|                                 |          | ingress, IPv6, remote_group_id: |
|                                 |          | 43a9ef6f-2cf2-484f-a37a-        |
|                                 |          | d45a290ac5db                    |
| b366a40b-4a47-4fbc-             | open-all | egress, IPv4                    |
| af51-4e214a29530d         

### 基本的な操作
操作の流れ。
* 仮想ルーターの作成 → 仮想ルーターと外部ネットワークを接続する（しなくても構いません。その場合は内部ネットワーク同士のルーティングに使います）
* 仮想ネットワークの作成 → 仮想サブネットの作成
* 仮想ルーターと仮想ネットワークを接続する（しなくても構いません。その場合は独立した単独ネットワークして使います）
* セキュリティグループの作成（今回の演習環境はあらかじめ open-all というセキュリティグループが作成されているのでスキップします）
* キーペアの作成（今回の演習環境はあらかじめ my-key というキーペアが作成されているのでスキップします）
* 仮想マシンの作成
* Floating IP の割り当て

#### 仮想ルーターの作成

自分用の仮想ルーターを作成し、外部ネットワーク（public/192.168.99.0/24）と接続します。

In [12]:
neutron router-create Ext-Router

Created a new router:
+-------------------------+--------------------------------------+
| Field                   | Value                                |
+-------------------------+--------------------------------------+
| admin_state_up          | True                                 |
| availability_zone_hints |                                      |
| availability_zones      |                                      |
| created_at              | 2017-05-21T09:28:42Z                 |
| description             |                                      |
| external_gateway_info   |                                      |
| flavor_id               |                                      |
| id                      | 7e53e749-5632-4f86-9cc8-db399d35d527 |
| name                    | Ext-Router                           |
| project_id              | 44be7b065a154e33aacbe7dbca18c6be     |
| revision_number         | 3                                    |
| routes                  |             

In [13]:
neutron router-list

+--------------------------------------+------------+-----------------------+
| id                                   | name       | external_gateway_info |
+--------------------------------------+------------+-----------------------+
| 7e53e749-5632-4f86-9cc8-db399d35d527 | Ext-Router | null                  |
+--------------------------------------+------------+-----------------------+


In [14]:
neutron router-gateway-set Ext-Router public

Set gateway for router Ext-Router


In [15]:
neutron router-list

+--------------------------------+------------+--------------------------------+
| id                             | name       | external_gateway_info          |
+--------------------------------+------------+--------------------------------+
| 7e53e749-5632-4f86-9cc8-db399d | Ext-Router | {"network_id": "17e4bba8-2a15  |
| 35d527                         |            | -46fb-ab54-fb463aee503c",      |
|                                |            | "enable_snat": true,           |
|                                |            | "external_fixed_ips":          |
|                                |            | [{"subnet_id": "a271b0a1-c912  |
|                                |            | -492c-858e-d5bd41888246",      |
|                                |            | "ip_address":                  |
|                                |            | "192.168.99.203"}]}            |
+--------------------------------+------------+--------------------------------+


#### 仮想ネットワークの作成
仮想ネットワークを作成して、この仮想ルーターとの接続を行います。この操作で、仮想ネットワークと public ネットワークが接続されます。

In [16]:
neutron net-create work-net

Created a new network:
+-------------------------+--------------------------------------+
| Field                   | Value                                |
+-------------------------+--------------------------------------+
| admin_state_up          | True                                 |
| availability_zone_hints |                                      |
| availability_zones      |                                      |
| created_at              | 2017-05-21T09:29:02Z                 |
| description             |                                      |
| id                      | 6e6eaff6-0f3b-4ed6-9e26-fad732dd0362 |
| ipv4_address_scope      |                                      |
| ipv6_address_scope      |                                      |
| mtu                     | 1450                                 |
| name                    | work-net                             |
| project_id              | 44be7b065a154e33aacbe7dbca18c6be     |
| revision_number         | 2          

In [17]:
neutron net-list

+---------------------------------+----------+---------------------------------+
| id                              | name     | subnets                         |
+---------------------------------+----------+---------------------------------+
| 17e4bba8-2a15-46fb-             | public   | a271b0a1-c912-492c-858e-        |
| ab54-fb463aee503c               |          | d5bd41888246                    |
| 6e6eaff6-0f3b-                  | work-net |                                 |
| 4ed6-9e26-fad732dd0362          |          |                                 |
+---------------------------------+----------+---------------------------------+


In [18]:
neutron subnet-create --ip-version 4 --gateway 172.16.11.254 --name work-subnet work-net 172.16.11.0/24

Created a new subnet:
+-------------------+--------------------------------------------------+
| Field             | Value                                            |
+-------------------+--------------------------------------------------+
| allocation_pools  | {"start": "172.16.11.1", "end": "172.16.11.253"} |
| cidr              | 172.16.11.0/24                                   |
| created_at        | 2017-05-21T09:29:17Z                             |
| description       |                                                  |
| dns_nameservers   |                                                  |
| enable_dhcp       | True                                             |
| gateway_ip        | 172.16.11.254                                    |
| host_routes       |                                                  |
| id                | 54718494-45cf-41e4-8a7e-c4c36ba90d0d             |
| ip_version        | 4                                                |
| ipv6_address_mode |        

In [19]:
neutron net-list

+---------------------------------+----------+---------------------------------+
| id                              | name     | subnets                         |
+---------------------------------+----------+---------------------------------+
| 17e4bba8-2a15-46fb-             | public   | a271b0a1-c912-492c-858e-        |
| ab54-fb463aee503c               |          | d5bd41888246                    |
| 6e6eaff6-0f3b-                  | work-net | 54718494-45cf-41e4-8a7e-        |
| 4ed6-9e26-fad732dd0362          |          | c4c36ba90d0d 172.16.11.0/24     |
+---------------------------------+----------+---------------------------------+


In [20]:
neutron router-interface-add Ext-Router work-subnet

Added interface 8bfa4a6b-aac1-4ffc-9fd6-a872d194fdd6 to router Ext-Router.


#### 仮想マシンを起動します。
この時、仮想マシンは先程作成した仮想ネットワークへ接続を行います。
Cirrosというテスト用のOSイメージを使って仮想マシンを作成します。この仮想マシンは非常にコンパクトでこのようなテストや演習に最適です。
本番環境で利用する際には、CentOSやRHELのイメージを登録しておき、そのイメージを利用します。

In [34]:
function get_uuid () { cat - | grep " id " | awk '{print $4}'; }
export MY_WORK_NET=`neutron net-show work-net | get_uuid`

nova boot --flavor m1.tiny --image "cirros-0.3.5" \
--key-name my-key --security-groups open-all \
--nic net-id=${MY_WORK_NET} \
test-vm-1

+--------------------------------------+-----------------------------------------------------+
| Property                             | Value                                               |
+--------------------------------------+-----------------------------------------------------+
| OS-DCF:diskConfig                    | MANUAL                                              |
| OS-EXT-AZ:availability_zone          |                                                     |
| OS-EXT-STS:power_state               | 0                                                   |
| OS-EXT-STS:task_state                | scheduling                                          |
| OS-EXT-STS:vm_state                  | building                                            |
| OS-SRV-USG:launched_at               | -                                                   |
| OS-SRV-USG:terminated_at             | -                                                   |
| accessIPv4                           |          

仮想マシンの起動を確認します。

In [35]:
nova list

+--------------------------------------+-----------+--------+------------+-------------+----------+
| ID                                   | Name      | Status | Task State | Power State | Networks |
+--------------------------------------+-----------+--------+------------+-------------+----------+
| b584bbde-38d3-41e5-a062-784b55d56aa0 | test-vm-1 | BUILD  | spawning   | NOSTATE     |          |
+--------------------------------------+-----------+--------+------------+-------------+----------+


ACTIVE状態になるまで待機します。この時、仮想マシンに割り当てられたIPアドレスをメモしておきます。

In [37]:
nova list

+--------------------------------------+-----------+--------+------------+-------------+----------------------+
| ID                                   | Name      | Status | Task State | Power State | Networks             |
+--------------------------------------+-----------+--------+------------+-------------+----------------------+
| b584bbde-38d3-41e5-a062-784b55d56aa0 | test-vm-1 | ACTIVE | -          | Running     | work-net=172.16.11.9 |
+--------------------------------------+-----------+--------+------------+-------------+----------------------+


#### Floating IPの割り当て
作成した仮想マシンは自分で作成した内部ネットワークに接続されており、仮想マシン→外部 の接続は可能ですが、外部→仮想マシンの接続は行なえません。
Floating IPを仮想マシンに割り当てて、外部→仮想マシンの接続が行えるようにします。
Floating IPとは、外部ネットワーク（ここではpublic）のIPアドレスを仮想マシンへNATするための機能です。

In [24]:
neutron floatingip-create public

Created a new floatingip:
+---------------------+--------------------------------------+
| Field               | Value                                |
+---------------------+--------------------------------------+
| created_at          | 2017-05-21T09:33:25Z                 |
| description         |                                      |
| fixed_ip_address    |                                      |
| floating_ip_address | 192.168.99.205                       |
| floating_network_id | 17e4bba8-2a15-46fb-ab54-fb463aee503c |
| id                  | 768f424b-216d-40e4-9511-5d9c2de7ecc7 |
| port_id             |                                      |
| project_id          | 44be7b065a154e33aacbe7dbca18c6be     |
| revision_number     | 1                                    |
| router_id           |                                      |
| status              | DOWN                                 |
| tenant_id           | 44be7b065a154e33aacbe7dbca18c6be     |
| updated_at          | 2017-

ここで表示されるUUIDをメモしておきます。

In [25]:
neutron floatingip-list

+------------------------+------------------+---------------------+---------+
| id                     | fixed_ip_address | floating_ip_address | port_id |
+------------------------+------------------+---------------------+---------+
| 768f424b-216d-         |                  | 192.168.99.205      |         |
| 40e4-9511-5d9c2de7ecc7 |                  |                     |         |
+------------------------+------------------+---------------------+---------+


ここで表示される 172.16.11.xx のIPを持つポートのUUIDをメモしておきます（仮想サーバーに割り当っているIP）

In [38]:
neutron port-list

+-----------------------+------+-------------------+-----------------------+
| id                    | name | mac_address       | fixed_ips             |
+-----------------------+------+-------------------+-----------------------+
| 681879b1-53c0-495c-   |      | fa:16:3e:ef:eb:c1 | {"subnet_id":         |
| 8ce1-3eb69bb189bb     |      |                   | "54718494-45cf-41e4   |
|                       |      |                   | -8a7e-c4c36ba90d0d",  |
|                       |      |                   | "ip_address":         |
|                       |      |                   | "172.16.11.9"}        |
| 8bfa4a6b-aac1-4ffc-   |      | fa:16:3e:ba:53:67 | {"subnet_id":         |
| 9fd6-a872d194fdd6     |      |                   | "54718494-45cf-41e4   |
|                       |      |                   | -8a7e-c4c36ba90d0d",  |
|                       |      |                   | "ip_address":         |
|                       |      |                   | "172.16.11.254"}      |

コマンドの引数として、 **Floating IPのUUID** **PortのUUID** の順で指定します

In [39]:
neutron floatingip-associate 768f424b-216d-40e4-9511-5d9c2de7ecc7 681879b1-53c0-495c-8ce1-3eb69bb189bb

Associated floating IP 768f424b-216d-40e4-9511-5d9c2de7ecc7


In [40]:
nova list

+--------------------------------------+-----------+--------+------------+-------------+--------------------------------------+
| ID                                   | Name      | Status | Task State | Power State | Networks                             |
+--------------------------------------+-----------+--------+------------+-------------+--------------------------------------+
| b584bbde-38d3-41e5-a062-784b55d56aa0 | test-vm-1 | ACTIVE | -          | Running     | work-net=172.16.11.9, 192.168.99.205 |
+--------------------------------------+-----------+--------+------------+-------------+--------------------------------------+


付与されたFloating IPへSSH接続します。
自分のノートPCのSSHか、仮想アプライアンスから実行してください。

この時に、/home/openstack/my-key.pem がログインするための秘密鍵になります（あらかじめ今回の演習用に作成して配置したものになります）

In [31]:
ssh -oStrictHostKeyChecking=no -i ~/my-key.pem cirros@192.168.99.205 hostname

test-vm-1


#### 後片付け
仮想マシンを削除し、割り当てたFloating IPも解除しておきます。

In [41]:
nova delete test-vm-1

Request to delete server test-vm-1 has been accepted.


In [42]:
nova list

+----+------+--------+------------+-------------+----------+
| ID | Name | Status | Task State | Power State | Networks |
+----+------+--------+------------+-------------+----------+
+----+------+--------+------------+-------------+----------+


以下で指定するUUIDは、上で作成したFloating IPのUUIDです。

In [43]:
neutron floatingip-delete 768f424b-216d-40e4-9511-5d9c2de7ecc7

Deleted floatingip(s): 768f424b-216d-40e4-9511-5d9c2de7ecc7


In [44]:
neutron floatingip-list